In [1]:
pip install python-docx pypdf

Note: you may need to restart the kernel to use updated packages.


In [10]:
from pathlib import Path
from docx import Document
import re

In [31]:
{
  "data/raw/law/주택임대차보호법/주택임대차보호법.docx": "a8f13c9e..."
}

{'data/raw/law/주택임대차보호법/주택임대차보호법.docx': 'a8f13c9e...'}

In [32]:
import hashlib

def get_file_hash(file_path: str) -> str:
    """파일 내용 기준 hash (sha256)"""
    sha = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            sha.update(chunk)
    return sha.hexdigest()

In [33]:
import json
from pathlib import Path

PROCESSED_LOG = Path("data/logs/processed_files.json")

def load_processed_log() -> dict:
    if PROCESSED_LOG.exists():
        with open(PROCESSED_LOG, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_processed_log(log: dict):
    PROCESSED_LOG.parent.mkdir(parents=True, exist_ok=True)
    with open(PROCESSED_LOG, "w", encoding="utf-8") as f:
        json.dump(log, f, ensure_ascii=False, indent=2)

In [34]:
def should_process(file_path: str, processed_log: dict) -> bool:
    current_hash = get_file_hash(file_path)

    # 처음 보는 파일
    if file_path not in processed_log:
        return True

    # 내용이 바뀐 파일
    if processed_log[file_path] != current_hash:
        return True

    # 이미 처리했고, 내용도 동일
    return False

In [42]:
def mark_as_processed(file_path: str, processed_log: dict):
    processed_log[file_path] = get_file_hash(file_path)

In [43]:
from pathlib import Path

RAW_ROOT = Path("data/raw")
ALLOWED_EXTENSIONS = {".docx", ".pdf"}

def collect_raw_files() -> list[str]:
    files = []
    for path in RAW_ROOT.rglob("*"):
        if path.is_file() and path.suffix.lower() in ALLOWED_EXTENSIONS:
            files.append(str(path))
    return files

In [44]:
processed_log = load_processed_log()
raw_files = collect_raw_files()  # 👈 이 줄이 핵심

for file_path in raw_files:
    if not should_process(file_path, processed_log):
        continue

    # 여기서 청킹
    print("청킹 대상:", file_path)

    mark_as_processed(file_path, processed_log)

save_processed_log(processed_log)

In [45]:
processed_log = load_processed_log()

for file_path in raw_files:
    if not should_process(file_path, processed_log):
        continue  # 👉 그냥 스킵

    # 🔥 여기서 청킹 실행
    chunks = chunk_file(file_path)

    # 처리 완료 표시
    mark_as_processed(file_path, processed_log)

save_processed_log(processed_log)

In [46]:
from pathlib import Path

def detect_source_type(file_path: str) -> str:
    path = Path(file_path)

    if "law" in path.parts:
        return "law"
    if "rule" in path.parts:
        return "rule"
    if "case" in path.parts:
        return "case"

    raise ValueError(f"source_type을 판별할 수 없는 경로: {file_path}")

In [47]:
Path("data/raw/law/주택임대차보호법/파일.docx").parts

('data', 'raw', 'law', '주택임대차보호법', '파일.docx')

In [49]:
processed_log = load_processed_log()
raw_files = collect_raw_files()

for file_path in raw_files:
    if not should_process(file_path, processed_log):
        continue

    source_type = detect_source_type(file_path)
    print(f"[{source_type.upper()}] 청킹 대상:", file_path)

    # 🔥 여기서 실제 청킹
    if source_type == "law":
        chunks = chunk_law(file_path)
    elif source_type == "rule":
        chunks = chunk_rule(file_path)
    elif source_type == "case":
        chunks = chunk_case(file_path)

    # ✅ 청킹이 끝났을 때만 기록
    mark_as_processed(file_path, processed_log)

save_processed_log(processed_log)

# law 구현

In [55]:
from docx import Document

def load_docx_text(file_path: str) -> str:
    doc = Document(file_path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    return "\n".join(paragraphs)

In [56]:
import re

ARTICLE_PATTERN = re.compile(r"(제\s*\d+조(?:의\d+)?)")

In [57]:
from pathlib import Path
import re

def chunk_law(file_path: str) -> list[dict]:
    text = load_docx_text(file_path)

    # 조문 기준 split (조문 제목을 유지하기 위해 lookahead 사용)
    parts = re.split(r"(?=제\s*\d+조(?:의\d+)?)", text)

    chunks = []
    law_name = Path(file_path).parent.name

    for part in parts:
        part = part.strip()
        if not part:
            continue

        # 조문 번호 추출
        match = re.match(r"(제\s*\d+조(?:의\d+)?)", part)
        if not match:
            continue

        article = match.group(1)

        chunk = {
            "id": f"law_{law_name}_{article}",
            "text": part,
            "metadata": {
                "source_type": "law",
                "law_name": law_name,
                "article": article,
                "source_path": file_path
            }
        }
        chunks.append(chunk)

    return chunks

In [58]:
chunks = chunk_law(file_path)

for c in chunks:
    print("=" * 40)
    print("ID:", c["id"])
    print("ARTICLE:", c["metadata"]["article"])
    print(c["text"][:300])  # 너무 길면 앞부분만

ID: law_rule_제1조
ARTICLE: 제1조
제1조(목적) 이 영은 「주택임대차보호법」에서 위임된 사항과 그 시행에 관하여 필요한 사항을 정함을 목적으로 한다.
[전문개정 2008. 8. 21.]
ID: law_rule_제2조
ARTICLE: 제2조
제2조(대항력이 인정되는 법인) 「주택임대차보호법」(이하 “법”이라 한다)
ID: law_rule_제3조
ARTICLE: 제3조
제3조제2항 후단에서 “대항력이 인정되는 법인”이란 다음 각 호의 법인을 말한다. <개정 2009. 9. 21., 2020. 9. 29.>
1. 「한국토지주택공사법」에 따른 한국토지주택공사(이하 “공사”라 한다)
2. 「지방공기업법」
ID: law_rule_제49조
ARTICLE: 제49조
제49조에 따라 주택사업을 목적으로 설립된 지방공사
[전문개정 2008. 8. 21.]
[
ID: law_rule_제1조의2
ARTICLE: 제1조의2
제1조의2에서 이동, 종전
ID: law_rule_제2조
ARTICLE: 제2조
제2조는
ID: law_rule_제8조
ARTICLE: 제8조
제8조로 이동  <2013. 12. 30.>]
ID: law_rule_제2조의2
ARTICLE: 제2조의2
제2조의2
[
ID: law_rule_제9조
ARTICLE: 제9조
제9조로 이동 <2013. 12. 30.>]
ID: law_rule_제3조
ARTICLE: 제3조
제3조(고유식별정보의 처리) 다음 각 호의 어느 하나에 해당하는 자는 법
ID: law_rule_제3조의6
ARTICLE: 제3조의6
제3조의6에 따른 확정일자 부여 및 임대차 정보제공 등에 관한 사무를 수행하기 위하여 불가피한 경우 「개인정보 보호법 시행령」
ID: law_rule_제19조
ARTICLE: 제19조
제19조제1호 및 제4호에 따른 주민등록번호 및 외국인등록번호를 처리할 수 있다. <개정 2016. 1. 22.>
1. 시장(「제주특별자치도 설치 및 국제자유도시 조성을 위한 특별법」
ID: law_rule_제11조
ART

# RULE 구현


In [59]:
import re

ARTICLE_PATTERN = re.compile(r"(제\s*\d+조(?:의\d+)?)")

In [61]:
from collections import defaultdict
from pathlib import Path

def chunk_rule(file_path: str) -> list[dict]:
    text = load_docx_text(file_path)

    # 조 번호별로 텍스트 누적
    article_map = defaultdict(list)

    # 줄 단위로 처리 (rule은 문맥 파편이 많아서 이게 안정적)
    lines = [line.strip() for line in text.split("\n") if line.strip()]

    current_article = None

    for line in lines:
        match = ARTICLE_PATTERN.search(line)
        if match:
            current_article = match.group(1)

        if current_article:
            article_map[current_article].append(line)

    rule_name = Path(file_path).parent.name
    chunks = []

    for article, contents in article_map.items():
        merged_text = "\n".join(contents)

        chunk = {
            "id": f"rule_{rule_name}_{article}",
            "text": merged_text,
            "metadata": {
                "source_type": "rule",
                "rule_name": rule_name,
                "article": article,
                "source_path": file_path
            }
        }
        chunks.append(chunk)

    return chunks

In [62]:
if source_type == "law":
    chunks = chunk_law(file_path)
elif source_type == "rule":
    chunks = chunk_rule(file_path)

NameError: name 'source_type' is not defined